# Laboratorio 1 - Minería de Datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sb
import sklearn.cluster as cluster
import sklearn.metrics as metrics
import sklearn.preprocessing
import scipy.cluster.hierarchy as sch
import skfuzzy as fuzz
import pylab
import sklearn.mixture as mixture
import pyclustertend 
import random

In [14]:
#importar csv
data = pd.read_csv('movies_2026.csv', encoding='latin-1')
pd.options.display.float_format = '{:,.0f}'.format

data.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount,releaseYear
0,1627085,0,Drama|Crime,NaN,NaN,NaN,NaN,0,95,False,...,2026-02-01,0,0,2,0,0,8,2,5,"2,026"
1,1626914,0,Animation,NaN,NaN,NaN,NaN,0,3,False,...,2026-02-01,0,0,1,0,0,4,0,0,"2,026"
2,1626898,0,Animation,NaN,NaN,NaN,NaN,0,2,False,...,2026-02-01,0,0,1,0,0,3,0,0,"2,026"
3,1626808,0,Thriller|Mystery|Documentary,NaN,NaN,NaN,NaN,0,5,False,...,2026-02-01,0,0,3,0,0,7,0,0,"2,026"
4,1626678,0,Animation,NaN,NaN,NaN,NaN,0,12,False,...,2026-02-01,0,0,1,0,0,3,0,0,"2,026"


1. Exploración rápida de sus datos

2. Tipo de cada una de las variables 

3. Investigue si las variables cuantitativas siguen una distribución normal y haga una
tabla de frecuencias

4. Preguntas

4.1.  ¿Cuáles son las 10 películas que contaron con más presupuesto?

In [15]:
top_10_budget = (
    data
    .dropna(subset=["budget"])
    .sort_values(by="budget", ascending=False)
    .head(10)
    [["title", "budget"]]
)

print(top_10_budget)

                                             title      budget
10606  Pirates of the Caribbean: On Stranger Tides 380,000,000
14600                      Avengers: Age of Ultron 365,000,000
15842                            Avengers: Endgame 356,000,000
5907                          Avatar: Fire and Ash 350,000,000
15843                       Avengers: Infinity War 300,000,000
14843                               Justice League 300,000,000
10053     Pirates of the Caribbean: At World's End 300,000,000
10497                             Superman Returns 270,000,000
17024                                The Lion King 260,000,000
13681                                      Tangled 260,000,000


4.2. ¿Cuáles son las 10 películas que más ingresos tuvieron?

In [16]:
top_10_revenue = (
    data
    .dropna(subset=["revenue"])
    .sort_values(by="revenue", ascending=False)
    .head(10)
    [["title", "revenue"]]
)

print(top_10_revenue)

                              title       revenue
13100                        Avatar 2,847,246,203
15842             Avengers: Endgame 2,797,800,564
10197                       Titanic 2,187,463,944
14837  Star Wars: The Force Awakens 2,068,223,624
15843        Avengers: Infinity War 2,046,239,637
8481                     Zootopia 2 1,744,338,246
14804                Jurassic World 1,671,713,208
17024                 The Lion King 1,667,635,327
18937       Spider-Man: No Way Home 1,631,853,496
13287                  The Avengers 1,518,815,515


4.3. ¿Cuál es la película que más votos tuvo?

In [19]:
top_1_voted = (
    data
    .dropna(subset=["voteCount"])
    .sort_values(by="voteCount", ascending=False)
    .head(1)
    [["title", "voteCount"]]
)

top_1_avg_vote= (
    data
    .dropna(subset=["voteAvg"])
    .sort_values(by="voteAvg", ascending=False)
    .head(1)
    [["title", "voteAvg"]]
)

print(top_1_voted)

print(top_1_avg_vote)

           title  voteCount
13401  Inception      30788
                   title  voteAvg
1813  GravaÃ§Ã£o Maldita       10


4.4.  ¿Cuál es la peor película de acuerdo a los votos de todos los usuarios? 

In [30]:
#se realiza una validación para ver que la película no tenga 0 votos,
#ya que si esta no ha sido evaluada no puede ser la peor película
#de esta manera busca eliminarse un sesgo

data_filtrada = data[
    (data["voteCount"] >= 50) &
    (data["voteAvg"].notna())
]

worst_movie = (
    data_filtrada
    .sort_values(by="voteAvg", ascending=True)
    .head(1)[["title", "voteAvg", "voteCount"]]
)

worst_movie


,title,voteAvg,voteCount
12656,Dragonball Evolution,3,1600


4.5.  ¿Cuántas películas se hicieron en cada año? ¿En qué año se hicieron más películas? Haga un gráfico de barras

4.6. ¿Cuál es el género principal de las 20 películas más recientes? ¿Cuál es el género
principal que predomina en el conjunto de datos? Represéntelo usando un gráfico. ¿A qué
género principal pertenecen las películas más largas?

4.7.  ¿Las películas de qué genero principal obtuvieron mayores ganancias?

4.8. ¿La cantidad de actores influye en los ingresos de las películas?¿se han hecho
películas con más actores en los últimos años?

4.9. ¿Es posible que la cantidad de hombres y mujeres en el reparto influya en la
popularidad y los ingresos de las películas?

4.10. ¿Quiénes son los directores que hicieron las 20 películas mejor calificadas?

4.11. ¿Cómo se correlacionan los presupuestos con los ingresos? ¿Los altos presupuestos significan altos ingresos? Haga los gráficos que necesite, histograma, diagrama de dispersión

4.12. ¿Se asocian ciertos meses de lanzamiento con mejores ingresos?

4.13.  ¿En qué meses se han visto los lanzamientos con mejores ingresos? ¿cuantas películas, en promedio, se han lanzado por mes?

4.14. ¿Cómo se correlacionan las calificaciones con el éxito comercial?

4.15. ¿Qué estrategias de marketing, como videos promocionales o páginas oficiales, generan mejores resultados?  

4.16. ¿La popularidad del elenco está directamente correlacionada con el éxito de taquilla?

5. Genere usted otras seis preguntas que le parezcan interesantes porque le permitan realizar otras exploraciones y respóndalas. No puede repetir ninguna de las instrucciones anteriores